# **Promedio de valor de Créditos para casas nueva y para vivienda usada**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_pachuca',
 'df_financiamientos_2020_pachuca',
 'df_financiamientos_2021_pachuca',
 'df_financiamientos_2022_pachuca',
 'df_financiamientos_2023_pachuca',
 'df_financiamientos_2024_pachuca']

## 2019

In [2]:
finance_2022 = df_financiamientos_2019_pachuca[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(1215, 4)
   id  mes  modalidad      monto
0   1    1          2   15000.00
1   2    1          2   31608.00
2   3    1          2   36018.00
3   4    1          1  130000.00
4   5    1          1  134542.22


In [3]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\219771975.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)


,id,mes,modalidad,monto,trimestre
3,4,1,Nueva,130000.00,1
4,5,1,Nueva,134542.22,1
5,6,1,Usada,140009.61,1
6,7,1,Nueva,152814.52,1
7,8,1,Nueva,186442.16,1
...,...,...,...,...,...
1210,1211,12,Usada,4840750.00,4
1211,1212,12,Nueva,6929939.00,4
1212,1213,12,Nueva,7488269.00,4
1213,1214,12,Nueva,11036275.00,4


In [4]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2019',
    2: '2T 2019',
    3: '3T 2019',
    4: '4T 2019'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2019,2T 2019,3T 2019,4T 2019
0,Nueva,"1,519,083.33","1,651,044.95","1,737,192.83","1,724,591.62"
1,Usada,"944,823.31","785,179.29","872,617.81","791,392.54"


In [5]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2019,"1,519,083.33"
4,Nueva,2T 2019,"1,651,044.95"
6,Nueva,3T 2019,"1,737,192.83"
8,Nueva,4T 2019,"1,724,591.62"
3,Usada,1T 2019,"944,823.31"
5,Usada,2T 2019,"785,179.29"
7,Usada,3T 2019,"872,617.81"
9,Usada,4T 2019,"791,392.54"


In [8]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos con símbolo de dólar
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el promedio con símbolo de dólar
        promedio_con_dolar = f'${df_modalidad.iloc[i]["promedio"]}'
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=promedio_con_dolar,
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2019', carpeta='assets\graficas')
fig.show()

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:64: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:64: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\4003868049.py:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\4003868049.py:64: SyntaxWarning:

invalid escape sequence '\g'



## 2020

In [9]:
finance_2022 = df_financiamientos_2020_pachuca[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(1178, 4)
   id  mes  modalidad    monto
0   1    1          2  10286.0
1   2    1          2  16471.0
2   3    1          2  25171.0
3   4    1          2  29856.0
4   5    1          2  46811.0


In [10]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
7,8,1,Usada,82200.03,1
9,10,1,Usada,117351.43,1
10,11,1,Usada,126176.36,1
11,12,1,Nueva,187499.00,1
12,13,1,Nueva,200998.49,1
...,...,...,...,...,...
1173,1174,12,Nueva,6099374.96,4
1174,1175,12,Nueva,6584000.00,4
1175,1176,12,Nueva,8904106.00,4
1176,1177,12,Nueva,13390156.00,4


In [11]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2020',
    2: '2T 2020',
    3: '3T 2020',
    4: '4T 2020'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2020,2T 2020,3T 2020,4T 2020
0,Nueva,"1,696,609.33","1,658,233.91","1,574,783.97","1,933,412.59"
1,Usada,"884,029.79","872,469.78","1,145,509.06","1,276,401.65"


In [12]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2020,"1,696,609.33"
4,Nueva,2T 2020,"1,658,233.91"
6,Nueva,3T 2020,"1,574,783.97"
8,Nueva,4T 2020,"1,933,412.59"
3,Usada,1T 2020,"884,029.79"
5,Usada,2T 2020,"872,469.78"
7,Usada,3T 2020,"1,145,509.06"
9,Usada,4T 2020,"1,276,401.65"


In [13]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos con símbolo de dólar
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el promedio con símbolo de dólar
        promedio_con_dolar = f'${df_modalidad.iloc[i]["promedio"]}'
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=promedio_con_dolar,
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2020', carpeta='assets\graficas')
fig.show()

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:63: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:63: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\4222299068.py:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\4222299068.py:63: SyntaxWarning:

invalid escape sequence '\g'



## 2021

In [14]:
finance_2022 = df_financiamientos_2021_pachuca[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(1430, 4)
   id  mes  modalidad     monto
0   1    1          2  20568.00
1   2    1          1  36070.86
2   3    1          2  38824.00
3   4    1          1  39449.85
4   5    1          2  63745.00


In [15]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
1,2,1,Nueva,36070.86,1
3,4,1,Nueva,39449.85,1
5,6,1,Usada,232000.90,1
6,7,1,Nueva,248284.47,1
7,8,1,Nueva,268813.82,1
...,...,...,...,...,...
1425,1426,12,Nueva,7640000.00,4
1426,1427,12,Usada,11719600.00,4
1427,1428,12,Nueva,13490590.00,4
1428,1429,12,Nueva,14009400.00,4


In [16]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2021',
    2: '2T 2021',
    3: '3T 2021',
    4: '4T 2021'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2021,2T 2021,3T 2021,4T 2021
0,Nueva,"1,726,001.29","1,640,809.94","1,776,954.58","1,856,235.46"
1,Usada,"1,142,443.24","1,135,746.00","1,229,093.93","1,186,562.79"


In [17]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2021,"1,726,001.29"
4,Nueva,2T 2021,"1,640,809.94"
6,Nueva,3T 2021,"1,776,954.58"
8,Nueva,4T 2021,"1,856,235.46"
3,Usada,1T 2021,"1,142,443.24"
5,Usada,2T 2021,"1,135,746.00"
7,Usada,3T 2021,"1,229,093.93"
9,Usada,4T 2021,"1,186,562.79"


In [18]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos con símbolo de dólar
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el promedio con símbolo de dólar
        promedio_con_dolar = f'${df_modalidad.iloc[i]["promedio"]}'
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=promedio_con_dolar,
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2021', carpeta='assets\graficas')
fig.show()

<>:48: SyntaxWarning:

invalid escape sequence '\g'

<>:64: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

<>:64: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\3567767405.py:48: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\3567767405.py:64: SyntaxWarning:

invalid escape sequence '\g'



## 2022

In [19]:
finance_2022 = df_financiamientos_2022_pachuca[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(1388, 4)
   id  mes  modalidad       monto
0   1    1          1        0.00
1   2    1          1  1563551.33
2   3    1          1  1263770.32
3   4    1          1  3614324.42
4   5    1          1   252907.18


In [20]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
1,2,1,Nueva,1563551.33,1
2,3,1,Nueva,1263770.32,1
3,4,1,Nueva,3614324.42,1
4,5,1,Nueva,252907.18,1
...,...,...,...,...,...
1378,1379,12,Usada,896000.00,4
1379,1380,12,Usada,843983.81,4
1380,1381,12,Usada,1089000.00,4
1381,1382,12,Usada,671476.15,4


In [21]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2022',
    2: '2T 2022',
    3: '3T 2022',
    4: '4T 2022'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2022,2T 2022,3T 2022,4T 2022
0,Nueva,"1,571,750.75","1,996,434.19","2,105,881.09","2,044,172.30"
1,Usada,"1,612,455.21","1,464,666.36","1,668,186.66","1,724,341.95"


In [22]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2022,"1,571,750.75"
4,Nueva,2T 2022,"1,996,434.19"
6,Nueva,3T 2022,"2,105,881.09"
8,Nueva,4T 2022,"2,044,172.30"
3,Usada,1T 2022,"1,612,455.21"
5,Usada,2T 2022,"1,464,666.36"
7,Usada,3T 2022,"1,668,186.66"
9,Usada,4T 2022,"1,724,341.95"


In [23]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos con símbolo de dólar
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el promedio con símbolo de dólar
        promedio_con_dolar = f'${df_modalidad.iloc[i]["promedio"]}'
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=promedio_con_dolar,
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2022', carpeta='assets\graficas')
fig.show()

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:65: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:65: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\2234423869.py:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\2234423869.py:65: SyntaxWarning:

invalid escape sequence '\g'



## 2023

In [24]:
finance_2022 = df_financiamientos_2023_pachuca[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(1399, 4)
   id  mes  modalidad       monto
0   1    1          1        0.00
1   2    1          1   486903.48
2   3    1          1  5702163.94
3   4    1          1  5284544.47
4   5    1          1   691154.29


In [25]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,0.00,1
1,2,1,Nueva,486903.48,1
2,3,1,Nueva,5702163.94,1
3,4,1,Nueva,5284544.47,1
4,5,1,Nueva,691154.29,1
...,...,...,...,...,...
1392,1393,12,Usada,881000.00,4
1393,1394,12,Usada,3817620.16,4
1394,1395,12,Usada,1740000.00,4
1395,1396,12,Usada,395789.35,4


In [26]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2023',
    2: '2T 2023',
    3: '3T 2023',
    4: '4T 2023'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2023,2T 2023,3T 2023,4T 2023
0,Nueva,"1,785,286.82","1,897,681.00","2,571,489.75","1,884,650.36"
1,Usada,"1,542,911.99","1,171,719.85","1,502,276.39","1,424,388.71"


In [27]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2023,"1,785,286.82"
4,Nueva,2T 2023,"1,897,681.00"
6,Nueva,3T 2023,"2,571,489.75"
8,Nueva,4T 2023,"1,884,650.36"
3,Usada,1T 2023,"1,542,911.99"
5,Usada,2T 2023,"1,171,719.85"
7,Usada,3T 2023,"1,502,276.39"
9,Usada,4T 2023,"1,424,388.71"


In [28]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos con símbolo de dólar
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el promedio con símbolo de dólar
        promedio_con_dolar = f'${df_modalidad.iloc[i]["promedio"]}'
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=promedio_con_dolar,
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2023', carpeta='assets\graficas')
fig.show()

<>:46: SyntaxWarning:

invalid escape sequence '\g'

<>:63: SyntaxWarning:

invalid escape sequence '\g'

<>:46: SyntaxWarning:

invalid escape sequence '\g'

<>:63: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\158346615.py:46: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\158346615.py:63: SyntaxWarning:

invalid escape sequence '\g'



## 2024

In [29]:
finance_2022 = df_financiamientos_2024_pachuca[['id','mes','modalidad','monto']]
shape = finance_2022.shape
print(shape)
print(finance_2022.head())

(792, 4)
   id  mes  modalidad       monto
0   1    1          1  5479808.78
1   2    1          1  1715027.31
2   3    1          1   938335.09
3   4    1          1  1014831.18
4   5    1          1  2344630.70


In [30]:
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    1: 'Nueva',
    3: 'Usada'
}
# Reemplazar los valores de 'modalidad' con el texto correspondiente utilizando replace
finance_2022['modalidad'] = finance_2022['modalidad'].replace(categorias)
# Convertir los valores que no pertenecen a 'Nueva' o 'Usada' a '-'
finance_2022.loc[~finance_2022['modalidad'].isin(['Nueva', 'Usada']), 'modalidad'] = '-'
# Eliminar los registros que tienen '-' en la columna 'modalidad'
finance_2022 = finance_2022[finance_2022['modalidad'] != '-']
# Asignar el trimestre al que pertenecen
finance_2022['trimestre'] = (finance_2022['mes'] - 1) // 3 + 1
finance_2022

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\219771975.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,modalidad,monto,trimestre
0,1,1,Nueva,5479808.78,1
1,2,1,Nueva,1715027.31,1
2,3,1,Nueva,938335.09,1
3,4,1,Nueva,1014831.18,1
4,5,1,Nueva,2344630.70,1
...,...,...,...,...,...
784,785,7,Usada,1253408.96,3
785,786,7,Usada,2718229.99,3
786,787,7,Usada,1633672.66,3
787,788,7,Usada,1596000.00,3


In [31]:
# Agrupar por 'modalidad' y 'mes', calcular el promedio del 'monto'
promedio_monto_por_modalidad_mes = finance_2022.groupby(['modalidad', 'trimestre'])['monto'].mean().unstack()
# Formatear los valores del promedio para que se muestren con comas para separar los miles y dos decimales
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.map('{:,.2f}'.format)

# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()

# Eliminar la columna 'trimestre'
#promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.drop(columns=['trimestre'])
# Cambiar los nombres de las columnas
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.rename(columns={
    1: '1T 2024',
    2: '2T 2024',
    3: '3T 2024',
    4: '4T 2024'
})
promedio_monto_por_modalidad_mes

trimestre,modalidad,1T 2024,2T 2024,3T 2024
0,Nueva,"2,015,805.09","2,295,956.45","2,651,405.94"
1,Usada,"1,431,903.90","1,597,460.38","1,513,424.70"


In [32]:
# Resetear el índice para convertir el DataFrame a un formato normal
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.reset_index()
# Utilizar el método melt para reorganizar el DataFrame
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.melt(
    id_vars=['modalidad'], 
    var_name='trimestre', 
    value_name='promedio'
)
# Ordenar los datos según la modalidad y el trimestre
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes.sort_values(by=['modalidad', 'trimestre'])
# Eliminar los registros que tienen index en la columna 'trimestre'
promedio_monto_por_modalidad_mes = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['trimestre'] != 'index']
promedio_monto_por_modalidad_mes

,modalidad,trimestre,promedio
2,Nueva,1T 2024,"2,015,805.09"
4,Nueva,2T 2024,"2,295,956.45"
6,Nueva,3T 2024,"2,651,405.94"
3,Usada,1T 2024,"1,431,903.90"
5,Usada,2T 2024,"1,597,460.38"
7,Usada,3T 2024,"1,513,424.70"


In [33]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# # Agregar etiquetas de texto a los puntos con símbolo de dólar
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el promedio con símbolo de dólar
        promedio_con_dolar = f'${df_modalidad.iloc[i]["promedio"]}'
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=promedio_con_dolar,
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2024', carpeta='assets\graficas')
fig.show()


<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:63: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:63: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\226055388.py:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14080\226055388.py:63: SyntaxWarning:

invalid escape sequence '\g'

